In [114]:
!cd container; ./build_and_push.sh gnn-inference

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  119.8kB
Step 1/23 : ARG REGION=us-west-2
Step 2/23 : FROM ubuntu:16.04
 ---> 96da9143fb18
Step 3/23 : LABEL com.amazonaws.sagemaker.capabilities.multi-models=true
 ---> Using cache
 ---> a913449a01cb
Step 4/23 : LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true
 ---> Using cache
 ---> a327e66b954c
Step 5/23 : RUN apt-get update &&     apt-get -y install --no-install-recommends     build-essential     ca-certificates     openjdk-8-jdk-headless     python3-dev     curl     vim     && rm -rf /var/lib/apt/lists/*     && curl -O https://bootstrap.pypa.io/get-pip.py     && python3 get-pip.py
 ---> Using cache
 ---> 4534f19902

In [84]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

# bucket = 'sagemaker-{}-{}'.format(region, account_id)
# prefix = 'demo-multimodel-endpoint'

role = get_execution_role()

In [85]:
# import models into hosting

from time import gmtime, strftime

model_name = 'DEMO-MultiModelModel' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
# model_url = 'https://s3-{}.amazonaws.com/{}/{}/'.format(region, bucket, prefix)
model_url = 's3://jdurago-insight-2020a/output/baseline/drug-prediction-gcn-200128-2135-015-1feea50f/output/'
container = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account_id, region, 'gnn-inference')

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container,
    'ModelDataUrl': model_url,
    'Mode': 'MultiModel'
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: DEMO-MultiModelModel2020-02-03-00-23-42
Model data Url: s3://jdurago-insight-2020a/output/baseline/drug-prediction-gcn-200128-2135-015-1feea50f/output/
Container image: 405787280505.dkr.ecr.us-west-2.amazonaws.com/gnn-inference:latest
Model Arn: arn:aws:sagemaker:us-west-2:405787280505:model/demo-multimodelmodel2020-02-03-00-23-42


In [86]:
# create endpoint configuration

endpoint_config_name = 'DEMO-MultiModelEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.t2.medium',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: DEMO-MultiModelEndpointConfig-2020-02-03-00-23-43
Endpoint config Arn: arn:aws:sagemaker:us-west-2:405787280505:endpoint-config/demo-multimodelendpointconfig-2020-02-03-00-23-43


In [87]:
# create endpoint
import time

endpoint_name = 'DEMO-MultiModelEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: DEMO-MultiModelEndpoint-2020-02-03-00-23-44
Endpoint Arn: arn:aws:sagemaker:us-west-2:405787280505:endpoint/demo-multimodelendpoint-2020-02-03-00-23-44
Endpoint Status: Creating
Waiting for DEMO-MultiModelEndpoint-2020-02-03-00-23-44 endpoint to be in service...


KeyboardInterrupt: 

# Invoke Models

In [115]:
import json

payload = 'CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)=[O+]2'
payload = json.dumps(payload)
# payload

In [141]:
%%time

import json

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    TargetModel='model.tar.gz', # this is the rest of the S3 path where the model artifacts are located
    Body=payload)

response
# print(*json.loads(response['Body'].read()), sep = '\n')

CPU times: user 3.92 ms, sys: 0 ns, total: 3.92 ms
Wall time: 30 ms


In [142]:
a = response['Body'].read()

In [143]:
a

b'[array([[9.800676e-23]], dtype=float32)]'

In [151]:
a

b'[array([[9.800676e-23]], dtype=float32)]'